# Estrutura de ficheiros utilizada para desenvolver o agente

### O agente está distribuído por 5 ficheiros:

+ `load.py` : Responsável pelo parser e a importação dos dados pelo csv.
+ Conjunto de restrições:
    + `hard_constraints_def.py` : Responsável por definir as funções que verificam as *hard constraints* que o agente tem de respeitar.
    + `soft_constraints_def.py` : Responsável por definir as funções que verificam as *soft constraints* que o agente pode, ou não, respeitar, mas é preferível que ele respeite.
+ `main.py` : Responsável por chamar todos os outros ficheiros e montar o agente em si utilizando a biblioteca **Constraint** de python. 


In [ ]:
def load_data_txt(path='dados/data.txt'):
    data = {
        'classes': {},           # #cc — cursos atribuídos a turmas
        'one_lesson_week': [],   # #olw — cursos com apenas uma aula/semana
        'teachers': {},          # #dsd — professores e UCs atribuídas
        'time_restrictions': {}, # #tr — restrições de horários (NÃO disponíveis)
        'room_restrictions': {}, # #rr — restrições de salas
        'online_classes': {},    # #oc — aulas online
        'time_availabilities': {} # calculadas automaticamente (disponíveis)
    }

    current_section = None

    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('—') or line.startswith('#head'):
                continue

            # Identificar secções
            if line.startswith('#'):
                if line.startswith('#cc'):
                    current_section = 'classes'
                elif line.startswith('#olw'):
                    current_section = 'one_lesson_week'
                elif line.startswith('#dsd'):
                    current_section = 'teachers'
                elif line.startswith('#tr'):
                    current_section = 'time_restrictions'
                elif line.startswith('#rr'):
                    current_section = 'room_restrictions'
                elif line.startswith('#oc'):
                    current_section = 'online_classes'
                else:
                    current_section = None
                continue

            parts = line.split()
            if not parts:
                continue

            if current_section == 'classes':
                turma = parts[0]
                ucs = parts[1:]
                data['classes'][turma] = ucs

            elif current_section == 'one_lesson_week':
                data['one_lesson_week'].append(parts[0])

            elif current_section == 'teachers':
                professor = parts[0]
                ucs = parts[1:]
                data['teachers'][professor] = ucs

            elif current_section == 'time_restrictions':
                professor = parts[0]
                not_available = [int(x) for x in parts[1:]]
                data['time_restrictions'][professor] = not_available

            elif current_section == 'room_restrictions':
                uc, sala = parts
                data['room_restrictions'][uc] = sala

            elif current_section == 'online_classes':
                uc, idx = parts
                data['online_classes'][uc] = int(idx)

    # Converter restrições em disponibilidades para todos os professores
    all_slots = set(range(1, 21))  # 20 blocos (5 dias × 4 blocos)
    for prof in data['teachers'].keys():  # garantir todos os professores
        unavailable = data['time_restrictions'].get(prof, [])  # se não tiver, assume []
        available = sorted(all_slots - set(unavailable))
        data['time_availabilities'][prof] = available

    return data


if __name__ == "__main__":
    dados = load_data_txt()

    print("=== Professores e UCs ===")
    for prof, ucs in dados['teachers'].items():
        print(f"{prof}: {ucs}")

    print("\n=== Slots indisponíveis ===")
    for prof, slots in dados['time_restrictions'].items():
        print(f"{prof}: {slots}")

    print("\n=== Slots disponíveis ===")
    for prof, slots in dados['time_availabilities'].items():
        print(f"{prof}: {slots}")


In [ ]:
APAGAAAAAAAAAAAAAAAAAAAAA# Ficheiro: data_config.py

from itertools import product
from load import *

dados = load_all()

# Configuração básica
dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
horas = [9, 11, 14, 16]
salas = [s['nome'] for s in dados['salas']]
turmas = [t['id'] for t in dados['turmas']]
ucs = [uc['id'] for uc in dados['unidades_curriculares']]
professores = [p['id'] for p in dados['professores']]

uc_to_turma = {}
uc_to_professor = {}

# Distribuir UCs pelas turmas
for i, uc in enumerate(dados['unidades_curriculares']):
    uc_id = uc['id']
    # Alternar entre turmas (0 → turma1, 1 → turma2, etc.)
    turma_index = i % len(turmas)
    uc_to_turma[uc_id] = turmas[turma_index]
    
    # Distribuir professores
    prof_index = i % len(professores)
    uc_to_professor[uc_id] = professores[prof_index]

# Disponibilidades do CSV
disponibilidades = dados['disponibilidades']

def check_professor_availability(prof_id):
    disponiveis = []
    for d, h in product(dias, horas):
        for disp in disponibilidades:
            if (disp['prof_id'] == prof_id and 
                disp['dia'] == d and 
                disp['hora_inicio'] <= h < disp['hora_fim']):
                disponiveis.append((d, h))
                break
    return disponiveis

In [ ]:
# Ficheiro: hard_constraints_def.py

# Conflitos de sala
def no_same_room_same_time(a1, a2):
    return not (a1[0] == a2[0] and a1[1] == a2[1])

def no_same_turma_same_time(a1, a2):
    return not (a1[0] == a2[0] and a1[3] == a2[3])

# Não permitir que um professor tenha duas aulas no mesmo horário
def no_same_professor_same_time(a1, a2):
    return not (a1[0] == a2[0] and a1[2] == a2[2])

# Máximo 3 aulas/dia por turma
def max_three_per_day_turma(*aulas):
    count_por_dia_turma = {}
    for aula in aulas:
        slot, _, _, turma, _ = aula
        dia = (slot - 1) // 4 + 1  
        key = f"{turma}_{dia}"
        count_por_dia_turma[key] = count_por_dia_turma.get(key, 0) + 1
        if count_por_dia_turma[key] > 3:
            return False
    return True

# Aulas da mesma UC em dias diferentes
def same_uc_different_days(a1, a2):
    if a1[4] != a2[4]:
        return True
    dia1 = (a1[0] - 1) // 4
    dia2 = (a2[0] - 1) // 4
    return dia1 != dia2

# Cada UC → 2 aulas
def exactly_two_per_uc(*aulas, ucs=None):
    count = {uc: 0 for uc in ucs}
    for aula in aulas:
        count[aula[4]] += 1
    return all(v == 2 for v in count.values())

# Cada turma → 10 aulas
def exactly_ten_per_turma(*aulas, turmas=None):
    count = {t: 0 for t in turmas}
    for aula in aulas:
        count[aula[3]] += 1
    return all(v == 10 for v in count.values())

In [ ]:
# Ficheiro: soft_constraints_def.py

# A mesma UC tem de ter aulas em dias diferentes
def check_distinct_day_classes(*aulas):
    uc_days = {}
    for aula in aulas:
        dia, _, _, _, _, uc = aula
        uc_days.setdefault(uc, set()).add(dia)
    return sum(len(dias) - 2 for dias in uc_days.values() if len(dias) > 2) == 0

# A mesma turma só pode ter até 4 dias de aulas
def check_weekly_days(*aulas):
    turma_days = {}
    for aula in aulas:
        dia, _, _, _, turma, _ = aula
        turma_days.setdefault(turma, set()).add(dia)
    return all(len(dias) <= 4 for dias in turma_days.values())

# As aulas da mesma UC não podem ser consecutivas
def check_consecutive_classes(*aulas):
    dia_to_horas_turma = {}
    for aula in aulas:
        dia, hora, _, _, turma, _ = aula
        dia_to_horas_turma.setdefault((turma, dia), []).append(hora)

    def hora_to_int(h):
        if isinstance(h, int):
            return h * 60
        elif isinstance(h, str):
            partes = h.split(':')
            if len(partes) == 2:
                hh, mm = map(int, partes)
                return hh * 60 + mm
            else:
                return int(h) * 60
        else:
            raise ValueError(f"Formato de hora inesperado: {h}")

    for (turma, dia), horas in dia_to_horas_turma.items():
        horas_ordenadas = sorted(hora_to_int(h) for h in horas)
        for i in range(1, len(horas_ordenadas)):
            if horas_ordenadas[i] - horas_ordenadas[i-1] != 60:
                return False
    return True

# Verifica se cada turma usa no máximo 3 salas diferentes nas aulas fornecidas
def check_different_classes(*aulas):
    turma_salas = {}
    for aula in aulas:
        _, _, sala, _, turma, _ = aula
        turma_salas.setdefault(turma, set()).add(sala)
    return all(len(salas) <= 3 for salas in turma_salas.values())



In [ ]:
# Ficheiro: main.py

from constraint import Problem
from load import load_data_txt
from hard_constraints_def import *
from soft_constraints_def import *

#  Carregar dados 
dados = load_data_txt()

dias = ["Segunda", "Terça", "Quarta", "Quinta", "Sexta"]
blocos = list(range(1, 21))
horas = [9, 11, 14, 16]

salas = ["Lab01", "Lab02", "Lab03"]

turmas = list(dados['classes'].keys())
ucs = [uc for ucs_list in dados['classes'].values() for uc in ucs_list]

# UC → Turma e UC → Professor
uc_to_turma = {}
uc_to_professor = {}
for turma, ucs_list in dados['classes'].items():
    for uc in ucs_list:
        uc_to_turma[uc] = turma
        for prof, prof_ucs in dados['teachers'].items():
            if uc in prof_ucs:
                uc_to_professor[uc] = prof
                break

#  Criar problema CSP 
problem = Problem()
all_vars = []

for uc in ucs:
    turma = uc_to_turma[uc]
    prof = uc_to_professor.get(uc)
    if not prof:
        print(f"⚠️ UC {uc} sem professor. Ignorada.")
        continue

    disp_slots = dados['time_availabilities'].get(prof, list(range(1, 21)))
    if not disp_slots:
        print(f"⚠️ UC {uc} do prof {prof} sem slots. Ignorada.")
        continue

    salas_validas = [dados['room_restrictions'][uc]] if uc in dados['room_restrictions'] else salas
    dominio = [(slot, s, prof, turma, uc) for slot in disp_slots for s in salas_validas]

    if not dominio:
        print(f"⚠️ UC {uc} sem domínios válidos. Ignorada.")
        continue

    # Variáveis separadas para cada aula da UC
    var1, var2 = f"UC{uc}_A1", f"UC{uc}_A2"
    problem.addVariable(var1, dominio)
    problem.addVariable(var2, dominio)
    all_vars.extend([var1, var2])

for i in range(len(all_vars)):
    for j in range(i + 1, len(all_vars)):
        problem.addConstraint(no_same_room_same_time, (all_vars[i], all_vars[j]))
        problem.addConstraint(no_same_turma_same_time, (all_vars[i], all_vars[j]))
        problem.addConstraint(no_same_professor_same_time, (all_vars[i], all_vars[j]))

for t in turmas:
    vars_t = [v for v in all_vars if uc_to_turma[v.split('_')[0][2:]] == t]
    if vars_t:
        problem.addConstraint(max_three_per_day_turma, vars_t)

for uc in ucs:
    var1, var2 = f"UC{uc}_A1", f"UC{uc}_A2"
    if var1 in all_vars and var2 in all_vars:
        problem.addConstraint(same_uc_different_days, (var1, var2))

problem.addConstraint(lambda *a: exactly_two_per_uc(*a, ucs=ucs), all_vars)
problem.addConstraint(lambda *a: exactly_ten_per_turma(*a, turmas=turmas), all_vars)

#  Gerar soluções iterativamente 
print("🧩 A gerar soluções válidas...")
MAX_SOLUTIONS = 200
solucoes = []

for sol in problem.getSolutionIter():
    solucoes.append(sol)
    if len(solucoes) >= MAX_SOLUTIONS:
        break

if not solucoes:
    print("❌ Nenhuma solução possível com os dados atuais")
    exit()

print(f"✅ Encontradas {len(solucoes)} soluções válidas (limitadas a {MAX_SOLUTIONS})")

#  Avaliar soft constraints 
def pontuacao(sol):
    aulas = []
    for val in sol.values():
        slot, sala, prof, turma, uc = val
        slot = int(slot)
        dia_index = (slot - 1) // 4
        bloco_index = (slot - 1) % 4
        dia = dias[dia_index]
        hora = horas[bloco_index]
        aulas.append((dia, hora, sala, prof, turma, uc))
    score = 0
    if check_distinct_day_classes(*aulas): score += 1
    if check_weekly_days(*aulas): score += 1
    if check_consecutive_classes(*aulas): score += 1
    if check_different_classes(*aulas): score += 1
    return score

avaliadas = [(sol, pontuacao(sol)) for sol in solucoes]
avaliadas.sort(key=lambda x: x[1], reverse=True)

melhor_sol, melhor_score = avaliadas[0]
print(f"🏆 Melhor solução encontrada com pontuação: {melhor_score}/4\n")

# === Visualizar melhor solução ===
for t in turmas:
    print(f"📘 Turma {t}")
    tabela = {dia: [""]*4 for dia in dias}
    for val in melhor_sol.values():
        slot, sala, prof, turma, uc = val
        if turma == t:
            dia_index = (slot-1)//4
            bloco_index = (slot-1)%4
            tabela[dias[dia_index]][bloco_index] = f"{uc} ({sala}, {prof})"

    print(f"{'Bloco':<6}" + ''.join(f"{d:<22}" for d in dias))
    for i in range(4):
        print(f"{i+1:<6}" + ''.join(f"{tabela[d][i]:<22}" for d in dias))
    print()

